In [126]:
import numpy as np
import plotly.graph_objects as go
from scipy.io import wavfile
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

def importWAV(filename):
    samplerate, rawData = wavfile.read(filename)
    
    time = np.linspace(0, rawData.shape[0]/samplerate, rawData.shape[0])   
    
    data = {'left':rawData[:, 0],'right':rawData[:, 1]}
    return time,data

In [127]:
v_100 = importWAV('40mV_100Hz.wav')
v_500 = importWAV('40mV_500Hz.wav')
v_1k = importWAV('40mV_1kHz.wav')
v_2k = importWAV('40mV_2kHz.wav')
v_4k = importWAV('40mV_4kHz.wav')
v_6k = importWAV('40mV_6kHz.wav')
v_8k = importWAV('40mV_8kHz.wav')
v_11k = importWAV('40mV_11kHz.wav')
v_13k = importWAV('40mV_13kHz.wav')
v_15k = importWAV('40mV_15kHz.wav')
v_18k = importWAV('40mV_18kHz.wav')

t_100 = v_100[0]
t_500 = v_500[0]
t_1k = v_1k[0]
t_2k = v_2k[0]
t_4k = v_4k[0]
t_6k = v_6k[0]
t_8k = v_8k[0]
t_11k = v_11k[0]
t_13k = v_13k[0]
t_15k = v_15k[0]
t_18k = v_18k[0]

t_arr = np.array([t_100, t_500, t_1k, t_2k, t_4k, t_6k, t_8k, t_11k, t_13k, t_15k, t_18k])

v_100 = v_100[1]['right']
v_500 = v_500[1]['right']
v_1k = v_1k[1]['right']
v_2k = v_2k[1]['right']
v_4k = v_4k[1]['right']
v_6k = v_6k[1]['right']
v_8k = v_8k[1]['right']
v_11k = v_11k[1]['right']
v_13k = v_13k[1]['right']
v_15k = v_15k[1]['right']
v_18k = v_18k[1]['right']

v_arr = np.array([v_100, v_500, v_1k, v_2k, v_4k, v_6k, v_8k, v_11k, v_13k, v_15k, v_18k])

f_arr = np.array([1E2, 5E2, 1E3, 2E3, 4E3, 6E3, 8E3, 11E3, 13E3, 15E3, 18E3])

C:\Users\logan\AppData\Local\Temp\ipykernel_24972\1836492366.py:8: WavFileWarning:

Chunk (non-data) not understood, skipping it.

C:\Users\logan\AppData\Local\Temp\ipykernel_24972\3374576093.py:25: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

C:\Users\logan\AppData\Local\Temp\ipykernel_24972\3374576093.py:39: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [128]:
V_amp_arr = np.sqrt(2) * np.array([29.24E-3, 28.25E-3, 28.25E-3, 28.25E-3, 28.25E-3, 29.23E-3,
                                   28.25E-3, 29.3E-3 ,29.19E-3, 29.19E-3, 29.18E-3])
V_amp_std = 0.02E-3

C_arr = np.array([])
C_std_arr = np.array([])

for v, f, V_amp in zip(v_arr, f_arr, V_amp_arr):
    n = round(6.5 * f)
    v_c = np.copy(v)
    v_sorted = np.sort(v_c)
    amps = (v_sorted[(-n - 1): -51] - v_sorted[50: n]) / 2
    C = np.mean(amps) / V_amp
    C_std = 1 / V_amp * np.sqrt((np.std(amps, ddof=1)) ** 2 + (np.mean(amps) * V_amp_std / V_amp) ** 2)
    
    C_arr = np.append(C_arr, C)
    C_std_arr = np.append(C_std_arr, C_std)

In [129]:
def T_series(x, a, b, c, d, e, f, g, h, j):
    return a + b*(x+c)**2 + j*(x+d)**3 + e*(x+f)**4 + g*(x+h)**5

def fourier_series(x, a0, an, bn, T):
    series = a0
    for n in range(1, len(an) + 1):
        series += an[n - 1] * np.cos(2 * np.pi * n * x / T) + bn[n - 1] * np.sin(2 * np.pi * n * x / T)
    return series

def T_F_series(x, a, b, c, d, e, f, g, h, j):
    return fourier_series(x, a0, an, bn, T) + a + b*(x+c)**2 + j*(x+d)**3 + e*(x+f)**4 + g*(x+h)**5

T = 18E3

fourier_coeffs = np.fft.fft(C_arr) / len(C_arr)

a0 = fourier_coeffs[0].real
an = 2 * fourier_coeffs[1:len(fourier_coeffs) // 2].real
bn = -2 * fourier_coeffs[1:len(fourier_coeffs) // 2].imag

p1, p1_cov = curve_fit(T_F_series, f_arr, C_arr, sigma=C_std_arr)
p2, p2_cov = curve_fit(T_series, f_arr, C_arr, sigma=C_std_arr)

f_space = np.linspace(0, 20E3, 1000)

In [130]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=f_arr, y=C_arr,
    name='Calculated C',
    mode='markers',
    error_y=dict(
        type='data',
        array=C_std_arr,
        color='blue'
    ),
    marker=dict(
        color='red',
        size=4
    )
))

fig.add_trace(go.Scatter(
    x=f_space, y=T_F_series(f_space, *p1),
    mode='lines',
    name='TS + FS'

))

fig.add_trace(go.Scatter(
    x=f_space, y=T_series(f_space, *p2),
    mode='lines',
    name='Fit'

))

fig.update_layout(
    xaxis_title='Frequency (Hz)',
    yaxis_title='Calibration Constant',
    xaxis=dict(
        type='linear'
    ),
    yaxis=dict(
        range=(6, 10)
    )
)


fig.show()
fig.write_image('SC vs F.png')